## Labelling the Dataframe Based on LDA model

In [1]:
import gensim
from gensim import corpora, models
import pandas as pd
import ast
from sklearn.model_selection import train_test_split
import random
from tqdm import tqdm


Load LDA Model

In [2]:
from gensim.models.ldamodel import LdaModel

# Load the saved LDA model
enron_lda = LdaModel.load("train_enron_lda_model.gensim")

# Use the loaded model
print(enron_lda.print_topics())

[(0, '0.014*"said" + 0.013*"power" + 0.010*"energy" + 0.007*"state" + 0.005*"electricity" + 0.005*"california" + 0.005*"new" + 0.005*"gas" + 0.004*"million" + 0.004*"market"'), (1, '0.010*"market" + 0.009*"ferc" + 0.009*"california" + 0.008*"final" + 0.006*"power" + 0.006*"capacity" + 0.006*"iso" + 0.005*"schedule" + 0.005*"transmission" + 0.005*"hour"'), (2, '0.012*"agreement" + 0.010*"deal" + 0.008*"gas" + 0.007*"know" + 0.006*"need" + 0.006*"let" + 0.005*"sara" + 0.005*"credit" + 0.005*"contract" + 0.005*"deals"'), (3, '0.011*"john" + 0.008*"david" + 0.008*"meeting" + 0.007*"communications" + 0.007*"database" + 0.007*"richard" + 0.006*"error" + 0.006*"communicationsenron" + 0.006*"james" + 0.006*"steve"'), (4, '0.006*"information" + 0.005*"new" + 0.005*"business" + 0.005*"time" + 0.004*"group" + 0.004*"need" + 0.004*"risk" + 0.003*"questions" + 0.003*"management" + 0.003*"meeting"'), (5, '0.016*"message" + 0.012*"original" + 0.010*"know" + 0.009*"get" + 0.006*"like" + 0.006*"let" + 

Load Topic Labels

In [3]:
# Define the mapping of topic numbers to labels
topic_labels = {
    0: "Energy Market & State-Level Power Issues",
    1: "Regulatory & Infrastructure (FERC, ISO, Transmission)",
    2: "Energy Trading & Legal Contracts",
    3: "Internal Communications & Personnel",
    4: "Corporate Strategy, Risk, & Operations",
    5: "Casual/Internal Email Chatter"
}

🔖 Topic 0
Top words: said, power, energy, state, electricity, california, market, gas, prices, stock

🟢 Label Suggestion:
"Energy Market & State-Level Power Issues"

📝 Rationale: Focuses on power, energy, electricity markets, particularly California and state-level utilities, including financial aspects like prices and stocks.

🔖 Topic 1
Top words: market, ferc, california, iso, capacity, transmission, rate, schedule, customers

🟢 Label Suggestion:
"Regulatory & Infrastructure (FERC, ISO, Transmission)"

📝 Rationale: Heavy presence of FERC, ISO, and terminology related to infrastructure (transmission, rates) and regulation.

🔖 Topic 2
Top words: agreement, deal, contract, credit, fax, legal, counterparty, master, isda, gas

🟢 Label Suggestion:
"Energy Trading & Legal Contracts"

📝 Rationale: Strong presence of trading and legal jargon — agreements, ISDA, deals, counterparties.

🔖 Topic 3
Top words: john, david, richard, meeting, database, communications, error, james, michael, scott

🟢 Label Suggestion:
"Internal Communications & Personnel"

📝 Rationale: Primarily names, suggesting internal discussions or emails; also includes terms like "communications", "database", and "error".

🔖 Topic 4
Top words: information, business, risk, management, data, review, project, group, system

🟢 Label Suggestion:
"Corporate Strategy, Risk, & Operations"

📝 Rationale: Business-centric, focusing on operations, management, projects, and risk — sounds like strategic planning or enterprise risk oversight.

🔖 Topic 5
Top words: message, original, know, get, kay, vince, friday, monday, think, want

🟢 Label Suggestion:
"Casual/Internal Email Chatter"

📝 Rationale: Very casual and informal. Lots of small talk and scheduling (e.g., days of the week, “get”, “know”, “let”), first names, and conversational tone.

Load DataFrame and Do Pre-Processing

In [4]:
def clean_bow(bow_list):
    return [word for word in bow_list if len(word) >= 3 and not any(char.isdigit() for char in word)]


In [6]:
enron_emails_df=pd.read_csv('../data/email_with_predicted_roles.csv')
enron_emails_df["BoW"] = enron_emails_df["BoW"].apply(ast.literal_eval)
enron_emails_df["cleaned_bow"] = enron_emails_df["BoW"].apply(clean_bow)
train_data, test_data = train_test_split(enron_emails_df["cleaned_bow"], test_size=0.2, random_state=42)

In [7]:
print(type(train_data))

<class 'pandas.core.series.Series'>


In [8]:
enron_dictionary = corpora.Dictionary(train_data)


In [10]:
print(enron_dictionary)

Dictionary<327253 unique tokens: ['big', 'commpany', 'electric', 'energy', 'gas']...>


In [ ]:
# Step 5: Evaluate 3 random samples from test data
sample_test_data = random.sample(list(test_data), 3)
for i, doc in enumerate(sample_test_data):
    bow = enron_dictionary.doc2bow(doc)
    #print(bow)
    topics = enron_lda.get_document_topics(bow)
    print(f"\nSample {i+1}: {doc}")
    # print(f"Assigned Topics: {topics}")
    print(f"Dominant Topic: Topic {max(topics, key=lambda x: x[1])[0]}")
    print(f"Dominant Topic Percentage: {round(max(topics, key=lambda x: x[1])[1],2)}")


[(145, 1), (163, 1), (172, 1), (200, 1), (565, 1), (589, 1), (668, 2), (787, 1), (803, 1), (837, 4), (854, 6), (866, 1), (953, 2), (1057, 1), (1078, 2), (1203, 1), (1249, 4), (1363, 5), (1424, 1), (1466, 2), (1596, 1), (1649, 1), (1804, 1), (1817, 1), (1820, 2), (2022, 1), (2102, 1), (2480, 1), (2700, 1), (2816, 1), (3066, 4), (3289, 1), (3344, 1), (3823, 1), (4145, 1), (4248, 1), (4519, 2), (5575, 4), (5891, 1), (6565, 1), (6688, 1), (6877, 4), (6879, 2), (6881, 3), (7038, 1), (7206, 1), (7345, 1), (8114, 1), (8407, 2), (9170, 1), (9491, 2), (10814, 1), (11293, 1), (12857, 1), (15925, 1), (15954, 2), (21849, 1), (22774, 1), (23865, 1), (33677, 1), (52298, 1), (113580, 1), (113581, 2), (113582, 2)]

Sample 1: ['november', 'enrons', 'board', 'directors', 'adopted', 'amendment', 'savings', 'plan', 'amendment', 'includes', 'following', 'changes', 'became', 'effective', 'november', 'savings', 'plan', 'contributions', 'deducted', 'november', 'paycheck', 'matched', 'cash', 'rather', 'stock',

In [14]:
from gensim.corpora.dictionary import Dictionary

# Step 1: Create dictionary from the entire dataset (cleaned_bow column)
# dictionary = Dictionary(enron_emails_df['cleaned_bow'])

# Step 2: Create corpus from each cleaned_bow entry
corpus = [enron_dictionary.doc2bow(text) for text in tqdm(enron_emails_df['cleaned_bow'], desc="generating BoW")]


generating BoW: 100%|██████████| 381581/381581 [00:20<00:00, 18235.27it/s]


In [15]:
# Get topic distribution for each document with tqdm
all_topics = [enron_lda.get_document_topics(doc, minimum_probability=0.0) for doc in tqdm(corpus, desc="Generating topic distributions")]


Generating topic distributions: 100%|██████████| 381581/381581 [01:36<00:00, 3966.76it/s]


In [16]:
# Get dominant topic and weight with tqdm
dominant_topics = []
dominant_percents = []

for topic_dist in tqdm(all_topics, desc="Extracting dominant topics"):
    dominant_topic = max(topic_dist, key=lambda x: x[1])
    dominant_topics.append(dominant_topic[0])
    dominant_percents.append(dominant_topic[1])

Extracting dominant topics: 100%|██████████| 381581/381581 [00:00<00:00, 1301230.54it/s]


In [19]:
print(dominant_percents[0])

0.5914403


In [20]:
enron_emails_df['Dominant_Topic'] = dominant_topics
enron_emails_df['Dominant_Topic_Pct'] = dominant_percents

In [25]:
enron_emails_df[["Dominant_Topic"]].value_counts()

Dominant_Topic
5                 111899
2                  99954
4                  90508
1                  31960
3                  31109
0                  16151
Name: count, dtype: int64

In [26]:
enron_emails_df

,From,To,Subject,X-cc,X-bcc,Content,Job_Title,Total_Sentence_Word_Count,From_Names,To_Names,Cleaned_Content,BoW,DateTime,cleaned_bow,Dominant_Topic,Dominant_Topic_Pct
0,richard.shapiro@enron.com,leslie.lawner@enron.com,Re: Talking points about California Gas market,"Christi L Nicolay, Joe Hartsoe, Rebecca W Cant...",X-Folder: \Phillip_Allen_Dec2000\Notes Folders...,"Leslie,after seeing point # 3 in writing , I w...","Vice President, Regulatory Affairs",133,richard shapiro,leslie lawner,leslieafter seeing point 3 writing extremely r...,"[leslieafter, seeing, point, 3, writing, extre...",12 Dec 2000,"[leslieafter, seeing, point, writing, extremel...",1,0.591440
1,keith.holst@enron.com,k..allen@enron.com,FW: must see,X-bcc:,"X-Folder: \PALLEN (Non-Privileged)\Allen, Phil...","-----Original Message-----\nFrom: \tMarquez, J...",Director,137,keith holst,k allen,original message marquez jose tuesday october ...,"[original, message, marquez, jose, tuesday, oc...",23 Oct 2001,"[original, message, marquez, jose, tuesday, oc...",3,0.644313
2,john.lavorato@enron.com,k..allen@enron.com,RE: Chase Backtest,X-bcc:,"X-Folder: \Phillip_Allen_Jan2002_1\Allen, Phil...",no\n\n -----Original Message-----\nFrom: \tAll...,"CEO, Enron America",164,john lavorato,k allen,original message allen phillip k monday decemb...,"[original, message, allen, phillip, k, monday,...",20 Dec 2001,"[original, message, allen, phillip, monday, de...",2,0.501085
3,louise.kitchen@enron.com,"wes.colwell@enron.com, georgeanne.hodges@enron...",NETCO,"Lavorato, John </O=ENRON/OU=NA/CN=RECIPIENTS/C...","X-Folder: \Phillip_Allen_Jan2002_1\Allen, Phil...",The New Year has arrived and we really to fina...,"President, Enron Online",375,louise kitchen,"wes colwell, georgeanne hodges, rob milnthorp",new year arrived finalize lot work regards mov...,"[new, year, arrived, finalize, lot, work, rega...",31 Dec 2001,"[new, year, arrived, finalize, lot, work, rega...",4,0.684726
4,louise.kitchen@enron.com,"tim.belden@enron.com, f..calger@enron.com, m.....",Re-start/Integration Planning,"Lavorato, John </O=ENRON/OU=NA/CN=RECIPIENTS/C...","X-Folder: \Phillip_Allen_Jan2002_1\Allen, Phil...",We have for the last couple of weeks started t...,"President, Enron Online",238,louise kitchen,"tim belden, f calger, m presto",last couple weeks started compile restartinteg...,"[last, couple, weeks, started, compile, restar...",27 Dec 2001,"[last, couple, weeks, started, compile, restar...",4,0.873879
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381576,john.zufferli@enron.com,kori.loibl@enron.com,Trade with John Lavorato,X-bcc:,"X-Folder: \ExMerge - Zufferli, John\Sent Items",This is a trade with OIL-SPEC-HEDGE-NG (John L...,Vice President,41,john zufferli,kori loibl,trade oilspechedgeng john lavoratos book john ...,"[trade, oilspechedgeng, john, lavoratos, book,...",28 Nov 2001,"[trade, oilspechedgeng, john, lavoratos, book,...",2,0.911955
381577,john.zufferli@enron.com,john.lavorato@enron.com,Gas Hedges,X-bcc:,"X-Folder: \ExMerge - Zufferli, John\Sent Items",Some of my position is with the Alberta Term b...,Vice President,23,john zufferli,john lavorato,position alberta term book send positions dire...,"[position, alberta, term, book, send, position...",28 Nov 2001,"[position, alberta, term, book, send, position...",2,0.640505
381578,john.zufferli@enron.com,dawn.doucet@enron.com,RE: CONFIDENTIAL,X-bcc:,"X-Folder: \ExMerge - Zufferli, John\Sent Items",2\n\n -----Original Message-----\nFrom: \tDouc...,"CEO, Enron America",49,john zufferli,dawn doucet,2 original message doucet dawn wednesday novem...,"[2, original, message, doucet, dawn, wednesday...",28 Nov 2001,"[original, message, doucet, dawn, wednesday, n...",5,0.858754
381579,john.zufferli@enron.com,jeanie.slone@enron.com,Calgary Analyst/Associate,X-bcc:,"X-Folder: \ExMerge - Zufferli, John\Sent Items",Analyst\t\t\t\t\tRank\n\nStephane Brodeur\t\t\...,Analyst Risk Management,18,john zufferli,jeanie slone,analyst rank stephane 

In [27]:
enron_emails_df["Label"] = enron_emails_df["Dominant_Topic"].map(topic_labels)


In [28]:
enron_emails_df.head()

,From,To,Subject,X-cc,X-bcc,Content,Job_Title,Total_Sentence_Word_Count,From_Names,To_Names,Cleaned_Content,BoW,DateTime,cleaned_bow,Dominant_Topic,Dominant_Topic_Pct,Label
0,richard.shapiro@enron.com,leslie.lawner@enron.com,Re: Talking points about California Gas market,"Christi L Nicolay, Joe Hartsoe, Rebecca W Cant...",X-Folder: \Phillip_Allen_Dec2000\Notes Folders...,"Leslie,after seeing point # 3 in writing , I w...","Vice President, Regulatory Affairs",133,richard shapiro,leslie lawner,leslieafter seeing point 3 writing extremely r...,"[leslieafter, seeing, point, 3, writing, extre...",12 Dec 2000,"[leslieafter, seeing, point, writing, extremel...",1,0.591440,"Regulatory & Infrastructure (FERC, ISO, Transm..."
1,keith.holst@enron.com,k..allen@enron.com,FW: must see,X-bcc:,"X-Folder: \PALLEN (Non-Privileged)\Allen, Phil...","-----Original Message-----\nFrom: \tMarquez, J...",Director,137,keith holst,k allen,original message marquez jose tuesday october ...,"[original, message, marquez, jose, tuesday, oc...",23 Oct 2001,"[original, message, marquez, jose, tuesday, oc...",3,0.644313,Internal Communications & Personnel
2,john.lavorato@enron.com,k..allen@enron.com,RE: Chase Backtest,X-bcc:,"X-Folder: \Phillip_Allen_Jan2002_1\Allen, Phil...",no\n\n -----Original Message-----\nFrom: \tAll...,"CEO, Enron America",164,john lavorato,k allen,original message allen phillip k monday decemb...,"[original, message, allen, phillip, k, monday,...",20 Dec 2001,"[original, message, allen, phillip, monday, de...",2,0.501085,Energy Trading & Legal Contracts
3,louise.kitchen@enron.com,"wes.colwell@enron.com, georgeanne.hodges@enron...",NETCO,"Lavorato, John </O=ENRON/OU=NA/CN=RECIPIENTS/C...","X-Folder: \Phillip_Allen_Jan2002_1\Allen, Phil...",The New Year has arrived and we really to fina...,"President, Enron Online",375,louise kitchen,"wes colwell, georgeanne hodges, rob milnthorp",new year arrived finalize lot work regards mov...,"[new, year, arrived, finalize, lot, work, rega...",31 Dec 2001,"[new, year, arrived, finalize, lot, work, rega...",4,0.684726,"Corporate Strategy, Risk, & Operations"
4,louise.kitchen@enron.com,"tim.belden@enron.com, f..calger@enron.com, m.....",Re-start/Integration Planning,"Lavorato, John </O=ENRON/OU=NA/CN=RECIPIENTS/C...","X-Folder: \Phillip_Allen_Jan2002_1\Allen, Phil...",We have for the last couple of weeks started t...,"President, Enron Online",238,louise kitchen,"tim belden, f calger, m presto",last couple weeks started compile restartinteg...,"[last, couple, weeks, started, compile, restar...",27 Dec 2001,"[last, couple, weeks, started, compile, restar...",4,0.873879,"Corporate Strategy, Risk, & Operations"


In [29]:
enron_emails_df.to_csv('../data/emails_with_topic_labels.csv')